In [54]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf
from regtabletotext import prettify_result

In [55]:
crsp_db = sqlite3.connect('../data/crsp.db.sqlite')

crsp_data = pd.read_sql(
    sql=("SELECT permno, gvkey, date, ret_excess, mktcap, "
         "mktcap_lag, exchange FROM crsp_monthly"), 
    con = crsp_db,
    parse_dates=['date']
    ).dropna()

compustat_db = sqlite3.connect('../data/compustat.db.sqlite')

compustat_data = pd.read_sql(
    sql ="SELECT at, act, che, lct, "
    "dlc, txp, dp, ib, gvkey, fyr, datadate "
    "FROM compustat",
    con = compustat_db,
    parse_dates=['datadate']
    ).dropna()

In [56]:
size = (crsp_data
  .query("date.dt.month == 12")
  .assign(yr=lambda x: (x["date"]+pd.DateOffset(months=4)).dt.year)
  .get(["gvkey", "yr", "mktcap"])
  .rename(columns={"mktcap": "size"})
)

Now we can can calculate earnings and accurals for each timestep. Note that the paper only uses companies with fiscal year end in December because of annual portfolios

In [57]:
compustat_lag = (compustat_data
    .query("fyr == 12")
    .assign(accural_lag = lambda x: x.act + x.dlc + x.txp - x.che - x.lct)
    .assign(datadate = lambda x: x.datadate + pd.DateOffset(years=1))
    .assign(at_lag = lambda x: x['at'])
    .get(['gvkey', 'datadate', 'accural_lag', "at_lag"])
)
compustat_cleaned_data = (compustat_data
    .merge(compustat_lag, how = "inner", on=['gvkey', 'datadate'])
    .assign(yr = lambda x: x.datadate.dt.year)
    .assign(accurals = lambda x: x.act + x.dlc + x.txp - x.che - x.lct - x.accural_lag - x.dp)
    .dropna()
)
compustat_cleaned_data['at'] = compustat_cleaned_data['at'].apply(lambda x: np.nan if x <= 0 else x)
compustat_cleaned_data['act'] = compustat_cleaned_data['act'].apply(lambda x: np.nan if x <= 0 else x).dropna()

In [58]:
compustat_cleaned_data = (compustat_cleaned_data
    .assign(avg_total_assets = lambda x: (x['at'] + x.at_lag)/2.0)
    .assign(earnings = lambda x: x.ib/x.avg_total_assets)
    .assign(accurals = lambda x: x.accurals/x.avg_total_assets)
    .assign(cash_flows = lambda x: x.earnings - x.accurals)
    .assign(sorted_date = lambda x: x.datadate + pd.DateOffset(months=4))
    .dropna()
)

In [59]:
compustat_cleaned_data.describe()

,at,act,che,lct,dlc,txp,dp,ib,fyr,datadate,accural_lag,at_lag,yr,accurals,avg_total_assets,earnings,cash_flows,sorted_date
count,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.0,184691,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691.000000,184691
mean,3263.824240,897.131584,280.107712,715.606130,159.653340,30.812497,151.121827,130.672449,12.0,2000-01-19 09:37:30.099896576,92.881930,3073.996229,1999.052109,-0.053230,3168.910234,-0.636733,-0.583502,2000-05-18 15:49:13.332864128
min,0.001000,0.001000,-40.000000,-0.002000,-882.000000,-66.000000,-3.233000,-44574.000000,12.0,1961-12-31 00:00:00,-65997.265000,0.000000,1961.000000,-1846.500000,0.000500,-4206.000000,-6401.714286,1962-04-30 00:00:00
25%,26.802500,12.878500,1.838000,6.145500,0.134000,0.000000,0.766000,-3.667500,12.0,1988-12-31 00:00:00,-0.838000,22.995500,1988.000000,-0.083959,25.726000,-0.088489,-0.066112,1989-04-30 00:00:00
50%,162.912000,68.941000,13.300000,32.477000,2.472000,0.024000,5.565000,1.814000,12.0,2000-12-31 00:00:00,4.880000,143.355000,2000.000000,-0.037903,155.136000,0.026826,0.053915,2001-04-30 00:00:00
75%,1132.839500,341.052000,82.178000,202.298500,22.132000,3.744500,44.972000,34.759000,12.0,2011-12-31 00:00:00,50.642000,1029.627000,2011.000000,0.002904,1084.389250,0.068593,0.114071,2012-04-30 00:00:00
max,663580.829000,264889.386000,139649.000000,227561.873000,122114.135000,17656.000000,52892.000000,76033.000000,12.0,2023-12-31 00:00:00,76482.947000,603595.692000,2023.000000,5245.571429,633588.260500,1230.000000,1328.000000,2024-04-30 00:00:00
std,15974.184073,4687.068776,1802.104665,4078.810591,1330.063199,215.806918,885.977856,1101.370120,0.0,NaN,1310.956685,15288.795527,14.885178,16.291378,15588.147094,24.613280,28.142240,NaN


In [60]:
compustat_cleaned_data[compustat_cleaned_data['earnings'] < -10000]

,at,act,che,lct,dlc,txp,dp,ib,gvkey,fyr,datadate,accural_lag,at_lag,yr,accurals,avg_total_assets,earnings,cash_flows,sorted_date


We can also assign portfolios

In [61]:
def assign_portfolio(data, sorting_variable, percentiles):
    """Assign portfolios to a bin according to a sorting variable."""
    
    breakpoints = (data
      .get(sorting_variable)
      .quantile(percentiles, interpolation="linear")
    )
    breakpoints.iloc[0] = -np.Inf
    breakpoints.iloc[breakpoints.size-1] = np.Inf
    
    assigned_portfolios = pd.cut(
      data[sorting_variable],
      bins=breakpoints,
      labels=pd.Series(range(1, breakpoints.size)),
      include_lowest=True,
      right=False
    )
    
    return assigned_portfolios
merged_data = (
    size
    .merge(compustat_cleaned_data, how = "inner", on = ["gvkey", "yr"])
)
merged_data = (merged_data
    .groupby('datadate')
    .apply(lambda x: x.assign(size_portfolios = assign_portfolio(merged_data, "size", [0, 0.5, 1]),
                              accrual_portfolios = assign_portfolio(merged_data,"accurals", np.linspace(0, 1, 11))), include_groups = True)
    .reset_index(drop=True)
)

C:\Users\joshu\AppData\Local\Temp\ipykernel_18460\3458522606.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(size_portfolios = assign_portfolio(merged_data, "size", [0, 0.5, 1]),


In [62]:
crsp_cleaned_data = (crsp_data
    .assign(sorted_date = lambda x: pd.to_datetime(
        x['date'].apply(lambda x: str(x.year) + "0430" if x.month >= 4 else str(x.year-1) + "0430")
        , format='%Y%m%d'))
)

In [63]:
merged_data = (crsp_cleaned_data
    .merge(merged_data, how = "inner", on = ['gvkey', 'sorted_date'])
    .dropna()
)

Sloan does adjustment based on the size factor. We can get this monthly from the ff3 database.

In [64]:
db = sqlite3.connect('../data/db.sqlite')

size_excess_return = pd.read_sql(
    sql = "SELECT smb,date from factors_ff5_monthly",
    con = db,
    parse_dates=['date']
)
size_excess_return = (size_excess_return
    .assign(sorted_date = lambda x: x['date'].apply(lambda x: pd.to_datetime(str(x.year) + "0430" if x.month >= 4 else str(x.year-1) + "0430", format='%Y%m%d')))
)
size_excess_return = size_excess_return.groupby('sorted_date').mean().reset_index().get(['sorted_date', 'smb'])

We adjust the returns for the size factor as Sloan does.

In [65]:
portfolios = (merged_data
    .get(["permno", "sorted_date", "accurals","earnings", "cash_flows", "size_portfolios", "accrual_portfolios", "ret_excess"])
    .merge(size_excess_return, how = "inner", on = ['sorted_date'])
    .assign(ret_excess = lambda x: x.apply(lambda x: ((x['ret_excess']-x["smb"]/12) if x["size_portfolios"] == 1 else x['ret_excess']), axis = 1))
)

In [66]:
portfolios.groupby(['size_portfolios', 'accrual_portfolios'])[['accurals','cash_flows','earnings']].mean()

C:\Users\joshu\AppData\Local\Temp\ipykernel_18460\3762512818.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  portfolios.groupby(['size_portfolios', 'accrual_portfolios'])[['accurals','cash_flows','earnings']].mean()


accurals  cash_flows  earnings
size_portfolios accrual_portfolios                                
1               1                  -0.249758   -0.035048 -0.284806
                2                  -0.113133    0.010938 -0.102195
                3                  -0.080107    0.012364 -0.067743
                4                  -0.059901   -0.005579 -0.065480
                5                  -0.044064   -0.002567 -0.046631
                6                  -0.030309   -0.016004 -0.046313
                7                  -0.016285   -0.024841 -0.041126
                8                   0.001937   -0.036337 -0.034400
                9                   0.033923   -0.068496 -0.034573
                10                  0.157927   -0.216056 -0.058129
2               1                  -0.211858    0.104330 -0.107528
                2                  -0.111841    0.091383 -0.020458
                3                  -0.079899    0.084044  0.004146
                4                  -0.059463    0.074707  0.015244
                5                  -0.044083    0.069902  0.025819
                6                  -0.030533    0.054345  0.023812
                7                  -0.016726    0.037146  0.020420
                8                   0.001443    0.016862  0.018305
                9                   0.031877   -0.015030  0.016847
                10                  0.125858   -0.093480  0.032379

We can see here that out adjusted returns based on size has done a good job in standardizing our values accross size portfolios.

In [67]:
portfolios.groupby(['accrual_portfolios'])[['accurals','cash_flows','earnings']].mean()

C:\Users\joshu\AppData\Local\Temp\ipykernel_18460\3526190707.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  portfolios.groupby(['accrual_portfolios'])[['accurals','cash_flows','earnings']].mean()


,accurals,cash_flows,earnings
accrual_portfolios,,,
1,-0.238651,0.005797,-0.232854
2,-0.112530,0.048490,-0.064039
3,-0.079992,0.051786,-0.028207
4,-0.059637,0.042823,-0.016814
5,-0.044076,0.043102,-0.000974
6,-0.030452,0.028857,-0.001595
7,-0.016552,0.012718,-0.003834
8,0.001671,-0.007655,-0.005985
9,0.033037,-0.045338,-0.012301


In [68]:
portfolios.groupby(['accrual_portfolios'])[['accurals','cash_flows','earnings']].median()

C:\Users\joshu\AppData\Local\Temp\ipykernel_18460\642992846.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  portfolios.groupby(['accrual_portfolios'])[['accurals','cash_flows','earnings']].median()


,accurals,cash_flows,earnings
accrual_portfolios,,,
1,-0.193194,0.116797,-0.091023
2,-0.110902,0.122303,0.012210
3,-0.079552,0.107213,0.027940
4,-0.059462,0.094867,0.035591
5,-0.044003,0.083323,0.039324
6,-0.030450,0.071110,0.040580
7,-0.016713,0.057494,0.040445
8,0.001271,0.042369,0.044250
9,0.031638,0.016658,0.049835


In [69]:
model = smf.ols("ret_excess ~  earnings", data=portfolios).fit()
prettify_result(model)

OLS Model:
ret_excess ~ earnings

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.009         0.0       52.737    0.000
earnings      0.001         0.0        3.169    0.002

Summary statistics:
- Number of observations: 1,204,107
- R-squared: 0.000, Adjusted R-squared: 0.000
- F-statistic: 10.043 on 1 and 1204105 DF, p-value: 0.002



In [70]:
model = smf.ols("ret_excess ~  accurals + cash_flows", data=portfolios).fit()
prettify_result(model)

OLS Model:
ret_excess ~ accurals + cash_flows

Coefficients:
            Estimate  Std. Error  t-Statistic  p-Value
Intercept      0.009       0.000       47.684      0.0
accurals      -0.011       0.002       -7.203      0.0
cash_flows     0.001       0.000        4.010      0.0

Summary statistics:
- Number of observations: 1,204,107
- R-squared: 0.000, Adjusted R-squared: 0.000
- F-statistic: 37.476 on 2 and 1204104 DF, p-value: 0.000



Much weaker effect than the paper.